In [ ]:
{
 "cells": [
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "Xw7YkEefehWo"
   },
   "source": [
    "# Домашнее задание. Классификация изображений",
    "",
    "Сегодня вам предстоить помочь телекомпании FOX в обработке их контента. Как вы знаете, сериал \"Симпсоны\" идет на телеэкранах более 25 лет, и за это время скопилось очень много видеоматериала. Персоонажи менялись вместе с изменяющимися графическими технологиями, и Гомер Симпсон-2018 не очень похож на Гомера Симпсона-1989. В этом задании вам необходимо классифицировать персонажей, проживающих в Спрингфилде. Думаю, нет смысла представлять каждого из них в отдельности.",
    ""
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "oG47vhLxKNln"
   },
   "source": [
    "### Установка зависимостей"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "WWgcwKwCLBfr",
    "outputId": "8d4c2f07-4cdb-409e-c5c7-1ddc6c1f1591"
   },
   "outputs": [],
   "source": [
    "# we will verify that GPU is enabled for this notebook",
    "# following should print: CUDA is available!  Training on GPU ...",
    "#",
    "# if it prints otherwise, then you need to enable GPU:",
    "# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU",
    "",
    "import torch",
    "import numpy as np",
    "import torchvision.transforms as transforms",
    "import PIL",
    "",
    "import pickle",
    "import numpy as np",
    "from skimage import io",
    "",
    "from tqdm import tqdm, tqdm_notebook",
    "from PIL import Image",
    "from pathlib import Path",
    "",
    "from torchvision import transforms",
    "from multiprocessing.pool import ThreadPool",
    "from sklearn.preprocessing import LabelEncoder",
    "from sklearn.model_selection import train_test_split",
    "from torch.utils.data import Dataset, DataLoader",
    "import torch.nn as nn",
    "",
    "from matplotlib import colors, pyplot as plt",
    "%matplotlib inline",
    "",
    "# в sklearn не все гладко, чтобы в colab удобно выводить картинки",
    "# мы будем игнорировать warnings",
    "import warnings",
    "warnings.filterwarnings(action='ignore', category=DeprecationWarning)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "xA3o2xC3MlMI",
    "outputId": "cbadb50a-9819-40b7-b52d-eba23329d79d"
   },
   "outputs": [],
   "source": [
    "# from google.colab import drive",
    "# drive.mount('/content/gdrive/')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "sxZ5llqziJ8k"
   },
   "outputs": [],
   "source": [
    "# !unzip -q /content/gdrive/MyDrive/journey-springfield.zip"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Аугментация изображений для тренировочного и тестового датасетов",
    "transform_train = transforms.Compose([",
    "    transforms.ToTensor(),",
    "    transforms.RandomHorizontalFlip(0.3),",
    "    transforms.RandomVerticalFlip(0.3),",
    "    transforms.RandomRotation(20),",
    "    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 3)),",
    "    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])",
    "])",
    "",
    "transform_test = transforms.Compose([",
    "    transforms.ToTensor(),",
    "    transforms.Normalize(",
    "        (0.485, 0.456, 0.406), ",
    "        (0.229, 0.224, 0.225)),",
    "])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "WTdzMtgJP15N",
    "outputId": "9e09ac5b-d349-433c-f101-9a135a12e99a"
   },
   "outputs": [],
   "source": [
    "# разные режимы датасета",
    "DATA_MODES = ['train', 'val', 'test']",
    "# все изображения будут масштабированы к размеру 224x224 px",
    "RESCALE_SIZE = 224",
    "# работаем на видеокарте",
    "",
    "train_on_gpu = torch.cuda.is_available()",
    "",
    "if not train_on_gpu:",
    "    print('CUDA is not available.  Training on CPU ...')",
    "    DEVICE = torch.device(\"cpu\")",
    "else:",
    "    print('CUDA is available!  Training on GPU ...')",
    "    DEVICE = torch.device(\"cuda\")"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "8ecnkB2xK1aE"
   },
   "source": [
    "Ниже мы исспользуем враппер над датасетом для удобной работы. Вам стоит понимать, что происходит с LabelEncoder и  с torch.Transformation.",
    "",
    "ToTensor конвертирует  PIL Image с параметрами в диапазоне [0, 255] (как все пиксели) в FloatTensor размера (C x H x W) [0,1] , затем производится масштабирование:",
    "$input = \\frac{input - \\mu}{\\text{standard deviation}} $, <br>       константы - средние и дисперсии по каналам на основе ImageNet",
    "",
    "",
    "Стоит также отметить, что мы переопределяем метод __getitem__ для удобства работы с данной структурой данных.",
    " Также используется LabelEncoder для преобразования строковых меток классов в id и обратно. В описании датасета указано, что картинки разного размера, так как брались напрямую с видео, поэтому следуем привести их к одному размер (это делает метод  _prepare_sample)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "cj32U5iTQUe4"
   },
   "outputs": [],
   "source": [
    "class SimpsonsDataset(Dataset):",
    "    \"\"\"",
    "    Датасет с картинками, который паралельно подгружает их из папок",
    "    производит скалирование и превращение в торчевые тензоры",
    "    \"\"\"",
    "    def __init__(self, files, mode, transform=transform_test):",
    "        super().__init__()",
    "        # список файлов для загрузки",
    "        self.files = sorted(files)",
    "        # режим работы",
    "        self.mode = mode",
    "",
    "        if self.mode not in DATA_MODES:",
    "            print(f\"{self.mode} is not correct; correct modes: {DATA_MODES}\")",
    "            raise NameError",
    "",
    "        self.transform = transform",
    "        ",
    "        self.len_ = len(self.files)",
    "",
    "        self.label_encoder = LabelEncoder()",
    "",
    "        if self.mode != 'test':",
    "            self.labels = [path.parent.name for path in self.files]",
    "            self.label_encoder.fit(self.labels)",
    "",
    "            with open('label_encoder.pkl', 'wb') as le_dump_file:",
    "                  pickle.dump(self.label_encoder, le_dump_file)",
    "",
    "    def __len__(self):",
    "        return self.len_",
    "",
    "    def load_sample(self, file):",
    "        image = Image.open(file)",
    "        image.load()",
    "        return image",
    "",
    "    def __getitem__(self, index):",
    "        x = self.load_sample(self.files[index])",
    "        x = self._prepare_sample(x)",
    "        x = np.array(x / 255, dtype='float32')",
    "        x = self.transform(x)",
    "        if self.mode == 'test':",
    "            return x",
    "        else:",
    "            label = self.labels[index]",
    "            label_id = self.label_encoder.transform([label])",
    "            y = label_id.item()",
    "            return x, y",
    "",
    "    def _prepare_sample(self, image):",
    "        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))",
    "        return np.array(image)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "j_odtTEzcaWH"
   },
   "outputs": [],
   "source": [
    "def imshow(inp, title=None, plt_ax=plt, default=False):",
    "    \"\"\"Imshow для тензоров\"\"\"",
    "    inp = inp.numpy().transpose((1, 2, 0))",
    "    mean = np.array([0.485, 0.456, 0.406])",
    "    std = np.array([0.229, 0.224, 0.225])",
    "    inp = std * inp + mean",
    "    inp = np.clip(inp, 0, 1)",
    "    plt_ax.imshow(inp)",
    "    if title is not None:",
    "        plt_ax.set_title(title)",
    "    plt_ax.grid(False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "QXyvJfT5jY-A"
   },
   "outputs": [],
   "source": [
    "#определим директории с тренировочными и тестовыми файлами",
    "TRAIN_DIR = Path('./train')",
    "TEST_DIR = Path('./testset')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "yUhzOq1zRJil"
   },
   "outputs": [],
   "source": [
    "train_val_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))",
    "test_files = sorted(list(TEST_DIR.rglob('*.jpg')))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "TmPhhKKlRyCF"
   },
   "outputs": [],
   "source": [
    "train_val_labels = [path.parent.name for path in train_val_files]",
    "train_files, val_files = train_test_split(train_val_files, test_size=0.25, \\",
    "                                          stratify=train_val_labels)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "aAimOLjSQGTh"
   },
   "outputs": [],
   "source": [
    "val_dataset = SimpsonsDataset(val_files, mode='val', transform=transform_test)"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "PmKSdyv1b7PD"
   },
   "source": [
    "Давайте посмотрим на наших героев внутри датасета."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 699
    },
    "id": "ltitWp3lXAZt",
    "outputId": "99d8d066-a9cf-420e-98fa-e4ac0d20086e"
   },
   "outputs": [],
   "source": [
    "fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(8, 8), \\",
    "                        sharey=True, sharex=True)",
    "for fig_x in ax.flatten():",
    "    random_characters = int(np.random.uniform(0,1000))",
    "    im_val, label = val_dataset[random_characters]",
    "    img_label = \" \".join(map(lambda x: x.capitalize(),\\",
    "                val_dataset.label_encoder.inverse_transform([label])[0].split('_')))",
    "    imshow(im_val.data.cpu(), \\",
    "          title=img_label,plt_ax=fig_x)"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "GpN9lSi4QVGt"
   },
   "source": [
    "Можете добавить ваши любимые сцены и классифицировать их. (веселые результаты можно кидать в чат)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "e2mk7MNtcUhJ"
   },
   "outputs": [],
   "source": [
    "def fit_epoch(model, train_loader, criterion, optimizer):",
    "    running_loss = 0.0",
    "    running_corrects = 0",
    "    processed_data = 0",
    "",
    "    for inputs, labels in train_loader:",
    "        inputs = inputs.to(DEVICE)",
    "        labels = labels.to(DEVICE)",
    "        optimizer.zero_grad()",
    "",
    "        outputs = model(inputs)",
    "        loss = criterion(outputs, labels)",
    "        loss.backward()",
    "        optimizer.step()",
    "        preds = torch.argmax(outputs, 1)",
    "        running_loss += loss.item() * inputs.size(0)",
    "        running_corrects += torch.sum(preds == labels.data)",
    "        processed_data += inputs.size(0)",
    "",
    "    train_loss = running_loss / processed_data",
    "    train_acc = running_corrects.cpu().numpy() / processed_data",
    "    return train_loss, train_acc"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "w_CD9--hcUjs"
   },
   "outputs": [],
   "source": [
    "def eval_epoch(model, val_loader, criterion):",
    "    model.eval()",
    "    running_loss = 0.0",
    "    running_corrects = 0",
    "    processed_size = 0",
    "",
    "    for inputs, labels in val_loader:",
    "        inputs = inputs.to(DEVICE)",
    "        labels = labels.to(DEVICE)",
    "",
    "        with torch.set_grad_enabled(False):",
    "            outputs = model(inputs)",
    "            loss = criterion(outputs, labels)",
    "            preds = torch.argmax(outputs, 1)",
    "",
    "        running_loss += loss.item() * inputs.size(0)",
    "        running_corrects += torch.sum(preds == labels.data)",
    "        processed_size += inputs.size(0)",
    "    val_loss = running_loss / processed_size",
    "    val_acc = running_corrects.double() / processed_size",
    "    return val_loss, val_acc"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "NaxYIwB3cUmX"
   },
   "outputs": [],
   "source": [
    "def train(train_files, val_files, model, epochs, batch_size, optimizer):",
    "    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)",
    "    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)",
    "",
    "    history = []",
    "    log_template = \"\Epoch {ep:03d} train_loss: {t_loss:0.4f} \\",
    "    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}\"",
    "",
    "    with tqdm(desc=\"epoch\", total=epochs) as pbar_outer:",
    "        criterion = nn.CrossEntropyLoss()",
    "",
    "        for epoch in range(epochs):",
    "            train_loss, train_acc = fit_epoch(model, train_loader, criterion, optimizer)",
    "            print(\"loss\", train_loss)",
    "",
    "            val_loss, val_acc = eval_epoch(model, val_loader, criterion)",
    "            history.append((train_loss, train_acc, val_loss, val_acc))",
    "",
    "            pbar_outer.update(1)",
    "            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\\",
    "                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))",
    "",
    "    return history"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "v6G7qbYqcUpL"
   },
   "outputs": [],
   "source": [
    "def predict(model, test_loader):",
    "    with torch.no_grad():",
    "        logits = []",
    "",
    "        for inputs in test_loader:",
    "            inputs = inputs.to(DEVICE)",
    "            model.eval()",
    "            outputs = model(inputs).cpu()",
    "            logits.append(outputs)",
    "",
    "    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()",
    "    return probs"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "yzwhB4K3dQOC",
    "outputId": "97d0a216-321d-44d3-d58b-8409eaeebdf5"
   },
   "outputs": [],
   "source": [
    "from torchvision.models import resnet50, resnet18",
    "",
    "n_classes = len(np.unique(train_val_labels))",
    "# Дообучим модель resnet18",
    "model = resnet18(pretrained=True)",
    "model.fc = nn.Sequential(",
    "    nn.Linear(512, 256),",
    "    nn.Dropout(),",
    "    nn.ReLU(),",
    "    nn.Linear(256, 128),",
    "    nn.Dropout(),",
    "    nn.ReLU(),",
    "    nn.Linear(128, 64),",
    "    nn.BatchNorm1d(64),",
    "    nn.ReLU(),",
    "    nn.Linear(64, 42),",
    ")",
    "    ",
    "model.to(DEVICE)",
    "print(model)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Заморозим первые 5 слоев сети, т.к. они выделяют низкоуровневые признаки, переобучение которых ",
    "# может ухудшить качество наших предсказаний",
    "",
    "counter = 0",
    "",
    "for child in model.children():",
    "    counter += 1",
    "    if counter < 5:",
    "        for param in child.parameters():",
    "            param.requires_grad = False",
    "print(counter)"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "bo3UND5RdgVg"
   },
   "source": [
    "Запустим обучение сети."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "WDkcxZ1kfD4a"
   },
   "outputs": [],
   "source": [
    "if val_dataset is None:",
    "    val_dataset = SimpsonsDataset(val_files, mode='val', transform=transform_test)",
    "",
    "train_dataset = SimpsonsDataset(train_files, mode='train', transform = transform_train)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "len(train_dataset)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "iDXoR8PIdfLD",
    "outputId": "aa45bc7b-7194-459b-e538-70b2726a3b56"
   },
   "outputs": [],
   "source": [
    "history = train(train_dataset, val_dataset, model=model, epochs=20, batch_size=128, optimizer=optimizer)"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "7qMAdL_BduXZ"
   },
   "source": [
    "Построим кривые обучения"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import gc",
    "from torch import cuda",
    "",
    "clear_mem = False",
    "",
    "if clear_mem == True:",
    "    gc.collect()",
    "    torch.cuda.empty_cache()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "2ryD_9yFdfNr"
   },
   "outputs": [],
   "source": [
    "loss, acc, val_loss, val_acc = zip(*history)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 773
    },
    "id": "GpQDWGkfdfQ5",
    "outputId": "f16bc2e5-fe5e-468d-ea38-a46dfb939d8b"
   },
   "outputs": [],
   "source": [
    "plt.figure(figsize=(15, 9))",
    "plt.plot(loss, label=\"train_loss\")",
    "plt.plot(val_loss, label=\"val_loss\")",
    "plt.legend(loc='best')",
    "plt.xlabel(\"epochs\")",
    "plt.ylabel(\"loss\")",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 773
    },
    "id": "pCs3gZoUvBBn",
    "outputId": "451369c0-656a-4fad-ede2-9cd48bc0d8a6"
   },
   "outputs": [],
   "source": [
    "plt.figure(figsize=(15, 9))",
    "plt.plot(acc, label=\"train_acc\")",
    "plt.plot([i.cpu().numpy() for i in val_acc], label=\"val_acc\")",
    "plt.legend(loc='best')",
    "plt.xlabel(\"epochs\")",
    "plt.ylabel(\"acc\")",
    "plt.show()"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "Gr9lRCJNNDfD"
   },
   "source": [
    "### Ну и что теперь со всем этим делать?"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "DSe0nQ-dJ8uy"
   },
   "source": [
    "![alt text](https://www.indiewire.com/wp-content/uploads/2014/08/the-simpsons.jpg)"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "y5k0UGeTNaQX"
   },
   "source": [
    "Хорошо бы понять, как сделать сабмит.",
    "У нас есть сеть и методы eval у нее, которые позволяют перевести сеть в режим предсказания. Стоит понимать, что у нашей модели на последнем слое стоит softmax, которые позволяет получить вектор вероятностей  того, что объект относится к тому или иному классу. Давайте воспользуемся этим."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "Z8PlF6o0N9O1"
   },
   "outputs": [],
   "source": [
    "def predict_one_sample(model, inputs, device=DEVICE):",
    "    \"\"\"Предсказание, для одной картинки\"\"\"",
    "    with torch.no_grad():",
    "        inputs = inputs.to(device)",
    "        model.eval()",
    "        logit = model(inputs).cpu()",
    "        probs = torch.nn.functional.softmax(logit, dim=-1).numpy()",
    "    return probs"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "pY_OoLoVO_9V"
   },
   "outputs": [],
   "source": [
    "random_characters = int(np.random.uniform(0,1000))",
    "ex_img, true_label = val_dataset[random_characters]",
    "probs_im = predict_one_sample(model, ex_img.unsqueeze(0))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "caivVFeAN9SY"
   },
   "outputs": [],
   "source": [
    "idxs = list(map(int, np.random.uniform(0,1000, 20)))",
    "imgs = [val_dataset[id][0].unsqueeze(0) for id in idxs]",
    "",
    "probs_ims = predict(model, imgs)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "t-0pRdHnQQKM"
   },
   "outputs": [],
   "source": [
    "label_encoder = pickle.load(open(\"label_encoder.pkl\", 'rb'))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "GNMFc7sfQh1a"
   },
   "outputs": [],
   "source": [
    "y_pred = np.argmax(probs_ims,-1)",
    "",
    "actual_labels = [val_dataset[id][1] for id in idxs]",
    "",
    "preds_class = [label_encoder.classes_[i] for i in y_pred]"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "iVePL0-BKHrF"
   },
   "source": [
    "Обратите внимание, что метрика, которую необходимо оптимизировать в конкурсе --- f1-score. Вычислим целевую метрику на валидационной выборке."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "_h-9dDWsKGU-",
    "outputId": "5baadcf0-f6a0-4a73-fd79-209045ad951e"
   },
   "outputs": [],
   "source": [
    "from sklearn.metrics import f1_score",
    "f1 = f1_score(actual_labels, y_pred, average='micro')",
    "",
    "print(\"F1-оценка:\", f1)"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "pxB9pfPfdCHr"
   },
   "source": [
    "Сделаем классную визуализацию,  чтобы посмотреть насколько сеть уверена в своих ответах. Можете исспользовать это, чтобы отлаживать правильность вывода."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 1000
    },
    "id": "VVjq4EC5ZZE7",
    "outputId": "cbe55ddb-c72e-4f41-ea6c-bd99fd9937fc"
   },
   "outputs": [],
   "source": [
    "import matplotlib.patches as patches",
    "from matplotlib.font_manager import FontProperties",
    "",
    "fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(12, 12), \\",
    "                        sharey=True, sharex=True)",
    "for fig_x in ax.flatten():",
    "    random_characters = int(np.random.uniform(0,1000))",
    "    im_val, label = val_dataset[random_characters]",
    "    img_label = \" \".join(map(lambda x: x.capitalize(),\\",
    "                val_dataset.label_encoder.inverse_transform([label])[0].split('_')))",
    "",
    "",
    "",
    "    imshow(im_val.data.cpu(), \\",
    "          title=img_label,plt_ax=fig_x)",
    "",
    "    actual_text = \"Actual : {}\".format(img_label)",
    "",
    "    fig_x.add_patch(patches.Rectangle((0, 53),86,35,color='white'))",
    "    font0 = FontProperties()",
    "    font = font0.copy()",
    "    font.set_family(\"fantasy\")",
    "    prob_pred = predict_one_sample(model, im_val.unsqueeze(0))",
    "    predicted_proba = np.max(prob_pred)*100",
    "    y_pred = np.argmax(prob_pred)",
    "",
    "    predicted_label = label_encoder.classes_[y_pred]",
    "    predicted_label = predicted_label[:len(predicted_label)//2] + '\' + predicted_label[len(predicted_label)//2:]",
    "    predicted_text = \"{} : {:.0f}%\".format(predicted_label,predicted_proba)",
    "",
    "    fig_x.text(1, 59, predicted_text , horizontalalignment='left', fontproperties=font,",
    "                    verticalalignment='top',fontsize=8, color='black',fontweight='bold')"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "hO9OLOMqIXRV"
   },
   "source": [
    "Попробуйте найти те классы, которые сеть не смогла расспознать. Изучите данную проблему, это понадобится в дальнейшем."
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "QEWTL6jgdh7L"
   },
   "source": [
    "### Submit на Kaggle"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "wrjQ6cxHIGtk"
   },
   "source": [
    "![alt text](https://i.redd.it/nuaphfioz0211.jpg)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "9UTbU0Zbc6Hb"
   },
   "outputs": [],
   "source": [
    "test_dataset = SimpsonsDataset(test_files, mode=\"test\")",
    "test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)",
    "probs = predict(model, test_loader)",
    "",
    "preds = label_encoder.inverse_transform(np.argmax(probs, axis=1))",
    "test_filenames = [path.name for path in test_dataset.files]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "_rTtbV1teD2k",
    "outputId": "2135cd45-a428-4924-a36a-13bc54855e3b"
   },
   "outputs": [],
   "source": [
    "! ls"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 206
    },
    "id": "yw0zZ-Hdd89s",
    "outputId": "82351b06-f0e6-4fa6-9cb4-b6ec5d127764"
   },
   "outputs": [],
   "source": [
    "import pandas as pd",
    "my_submit = pd.read_csv(\"./sample_submission.csv\")",
    "my_submit = pd.DataFrame({'Image_id': test_filenames, 'Expected': preds})",
    "my_submit.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "VIYaqa20iYTL"
   },
   "outputs": [],
   "source": [
    "# TODO : сделайте сабмит (это важно, если Вы не справляетесь, но дошли до этой ячейки, то сообщите в чат и Вам помогут)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "5rdlyMKtiYe2"
   },
   "outputs": [],
   "source": [
    "my_submit.to_csv('resnet18.csv', index=False)"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {
    "id": "h3M9SQZ7MuUq"
   },
   "source": [
    "## Приключение?",
    "",
    "А теперь самое интересное, мы сделали простенькую сверточную сеть и смогли отправить сабмит, но получившийся скор нас явно не устраивает. Надо с этим что-то сделать.",
    "",
    "Несколько срочныйх улучшейни для нашей сети, которые наверняка пришли Вам в голову:",
    "",
    "",
    "*   Учим дольше и изменяем гиперпараметры сети",
    "*  learning rate, batch size, нормализация картинки и вот это всё",
    "*   Кто же так строит нейронные сети? А где пулинги и батч нормы? Надо добавлять",
    "*  Ну разве Адам наше все? [adamW](https://www.fast.ai/2018/07/02/adam-weight-decay/) для практика, [статейка для любителей](https://openreview.net/pdf?id=ryQu7f-RZ) (очень хороший анализ), [наши ](https://github.com/MichaelKonobeev/adashift/) эксперименты для заинтересованных.",
    "",
    "* Ну разве это deep learning? Вот ResNet и Inception, которые можно зафайнтьюнить под наши данные, вот это я понимаю (можно и обучить в колабе, а можно и [готовые](https://github.com/Cadene/pretrained-models.pytorch) скачать).",
    "",
    "* Данных не очень много, можно их аугументировать и  доучититься на новом датасете ( который уже будет состоять из, как  пример аугументации, перевернутых изображений)",
    "",
    "* Стоит подумать об ансамблях",
    "",
    "",
    "Надеюсь, что у Вас получится!",
    "",
    "![alt text](https://pbs.twimg.com/profile_images/798904974986113024/adcQiVdV.jpg)"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Kaggle Score"
   ]
  },
  {
   "attachments": {
    "31a22949-a3a9-47d7-8b17-39f1e51b18df.png": {
     "image/png": "iVBORw0KGgoAAAANSUhEUgAABe4AAABsCAYAAAAcwSKzAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsMAAA7DAcdvqGQAACgZSURBVHhe7d0NdFTlve/xHySEQAgEEkCSoIkpJoLyokbFBDSx9gK2SClgr4RasO2B1FMp9yC3mmDbxPYGWgoeD6S14LFAbTFFSFvhSCUqIGqgQkQNKiZgEgQTCISXkBe4e888k5nJCyQByWC/n7Vm5f88s2fvPXssXes3z/x3p8iBUecEAAAAAAAAAAB8QmfzFwAAAAAAAAAA+ACCewAAAAAAAAAAfAjBPQAAAAAAAAAAPqTFHvdRMTGmAgAAAAAAAAAA7VW8b5+z6NpdOnPKWZ8HN6cFAAAAAAAAAOAL5P9ckbMoL9G50o907p1XdHZrjlR7xjnfCK1yAAAAAAAAAAC4HMIi1WlYkjp/N1OdJ86RunQ1T3gjuAcAAAAAAAAA4DLrPO4H6pw4yYy8EdwDAAAAAAAAANABOo2421TeCO4BAAAAAAAAAOgAnSIGmcobwT0AAAAAAAAAAB0hLNIU3gjuAQAAAAAAAADwIQT3AAAAAAAAAAD4EIJ7AAAAAAAAAAB8CME9AAAAAAAAAAA+hOAeAAAAAAAAAAAfQnAPAAAAAAAAAIAPIbgHAAAAAAAAAMCHENwDAAAAAAAAAOBDCO4BAAAAAAAAAPAhBPcAAAAAAAAAAPgQgnsAAAAAAAAAAHwIwT0AAAAAAAAAAD6E4B4AAAAAAAAAAB9CcA8AAAAAAAAAgA8huAcAAAAAAAAAwIcQ3AMAAAAAAAAA4EMI7gEAAAAAAAAA8CEE9wAAAAAAAAAA+BCCewAAAAAAAAAAfAjBPQAAAAAAAAAAPoTgHgAAAAAAAAAAH9IpcmDUOVMDl134mHlKnxGvvo5RkV6aMlcrHLVLhMbMnK0pybGKHtBXXWs+V1lxkfI3LVPWnwrNNgAAAAAAAADgu/yfKzJVU3UPRpvK7coM7nv5qXOI948FzlXW6twxM4Bv+P5C5dzj/I/uk02T9OgzjtKI0OSMJUodF61gM2MH9+tvnqQMM9LAaVqyLFUJAwLMhLeK/KV6ZOZyEd8DAAAAAAAA8GVfzuC+VzcFTAxVt8F91DWsmzr5mfnG6utUX35c1e8f0um1x1VPkN+xnsjRjvHO/+iKc2/WpJ85Sss0LfhbqpJdgXyd9fC3C+/gfsby7Uod7tym5mCBtr5WqIqe4Yoflagok/YX58609pvvHAAAAAAAAACAD2prcO/TPe473RCm4IXD1e8/h6p3UoQC+58ntLf5+cuvfx8FJV2vsP+MV9jCa9T1Btr4+54oXWuH9nVVKsxJ0+J3qsy8p1QlXmeC/Yqtyvr6dD26MEtZ6Y9o0rQ1KrTDfkvUiPsU7iwBAAAAAAAA4EvBN1PtyJ7q/oub1O/RGHXv31WdzHTbdJZf/6sU8mi8+v4iQl0izbSvmZqtvJ07tcN65DyRqHkrX3XUO3bmKN1sIiVq9lM5ytvu3M7x2P6qcp6arQSzhVuExszJVu6rHttajzdeWq3Mb8eZbWzTtMy1zV/SFT4pUzmveLzmlRwt+L7n9i6tORezb7Pa3hY13rltzhNmoqJAq56YqpRfbjATjYWqZ3dnVXOoSOudpdOnZao6bWr/rtY7bp0Eu3XPK9vd52098uzrMqnxHlpzDSM074+u53K14DYz3cD7+cwmzwMAAAAAAABA83wuuO907zUKy7hewZFdzIy3c9W1qi05rNMfNnqUnFR99VmzlbfOkZHqkzFUwfc2v09fEXzbfE0e7O747pSo9L8sVEpCtII9W70HBCsqYZoW/iXdK7y/71fPKnNqvMLNbmpO1Tj+BvSP05i52Vr2nWZi7q7D9NT/GauoEDO2hUQreeYCLRhtxg5tO5eWrdCPHpquxRtLzbg5G1X4qbMKiBqm1IHO2mH0UEWY91fxyTa1plFOwhM5Wjgz2XqPniduXXP7uvxktdd1ad01LFXWDld3/QjFTYw3tTEwRTfGmPrTAq1/y9QAAAAAAAAAcAE+FNx3lt+sG9T3/qvk17gdTu1pVW//SOVz83X4B//UkceKdDyz0eOxPSr/gfX83D2q3H5EtbXmtS5+3dT9/uHqMyuonSv4v3ih/UNVtW+zVixMU9rCVdpszd33m/m6L8oOm2tUlpellJtv1i03T1VWXqk1Y4faEzTvSVdoPFuTR4U6y0NblTXhZt0xaqRu+fEaFZ6yJ4M1LDnF8bSXAdEKP7RZS9Ot46Yv1eZPnUG1HUjHPzDN1G05l5WadZf1XK67b5Pd4/4Wa3tnn/tSlZlQvmX5Slu0TsX2jrsP1Yw/vqxVT2VqQXaO8hYmO9vjVORr5cJ1dnV+45do/vhoOSL7qkKt/9lU61zGa9aSzSpzvNVgxc9Il/PKtOEaLtqsAsecFD5kijyj+/BvD1Wco2+/VLhtWau+XAAAAAAAAAAAm88E936zBitsZKNQvf60Tv+1QIdnFujYsiOqP9T8inpP5w6d1JllH+nIzHyV//Ww6uvNEw6d1WXkDeo9q5sZ+5i9azR1ylwt/dMGbfzTOm3TNI0ZYULkves08z/WyLnGu1Av/EeGNpvwOzw+RWMcVbACTVisyjJtc4Xjr2fp0TkzHUH7Hd/NMpMeThVqzb/P1YqXrOO+tFyP/vu6hh7ywX2jnEWbz+USeD1DacvWqbjSqruHKi5hrJLjoxVsvceag1u19PEMrbrgFwBSyrhhcp55jQpWT1VGrn3mpcr/w1xl/KNINZWlKiwslW60t2nLNVyurR+aLzkGxGpyQzucCE0bYVrq1BVqx8Lz/bIAAAAAAAAAALz5RHDvN/0GhY4MMiOnc+WlOpJeoOMvnNa5xqvnW6P2rOpfKFJ5+gc6We4d+NvhfZ/pvhfel+1bpTJTOwyMbWgJU+UfptSMTGU2PO5TcL0JjUMjzGrvHfrkkKOQYqco95Vc5SxfovSHpyjO2rOrsUsTFUVa4xmAe/aQd2nzuVysCKX85mWteGSCs4XPqQoV7yuyHqWqqpMCBiQqNdu7xU3zIhR3lTnxU4Xa8YyzdMlPn6Q77h6vlJkZWvWuPdO2a7hi/duqcFQe7XI82uTU7Nmsxc4SAAAAAAAAAFql44P7ERHqleS90v7s/iKVzytRbYmZuBglx3ViXoGO7TdLyB06q0tSjLqPMEMfUVPXaGX26DCZyFnBMckaM26s1yPB0bbG0wY9+v/WqNBeoW4LiVDU8ETdN32eFmTn6o2/ZWu2V896o/6M9xcGzWnzuVyk8Y9q2uhQR3ubmuJ1emTU1zRpyiTrMV5J05eqoMreKFjxsxZotmf/+yaSFerq3W+9z+OmbFkbr2Fungqdyb3CRzh/beBuk1OjwjeX2wUAAAAAAAAAtFrHBvdduiloRqQ8bxlrh/YVPz+ss+1ZZd+S2jOq/vmeRuF9kIKtY/v58v1qP66SI5+2VGxbrDRHD/rmHov1gtnObumScvfNSvnxUr3wj3wVFlc4VqjbAgbEK2VOZvtWxLfnXC7CmKTrG9rbFG7K0DZHbby/XGveMWl5wLUaeq+zbF6hqk6YsrXadA3XadU75guX0Ggl3+bRJqfiba1vtMIfAAAAAAAAAC6kQ4P7zg9cqx69zMBW/ZkqL3Vo7+II7z/S6WoztvWKUM8HfDi5f6tI5ebmp6Fh4Spw9KB3Pwo+L1exo97qbuEyMF7J356gqKLlypo3Uynf+pqSJqVps6v9y8C49vWgb8+5XITgrq4V/AEK6G5KD2Hd3M/3NK33m5ev4nLTxif4WsWPd5Yu4XOeVe5fVmtZ1mxNdq3cb+M1zF9bYH6xYLfLmdHQJqfivY1a7ywBAAAAAAAAnxJ3/eBL8sAXo+OC+y491WN0DzOwndHJ5/ar9osI7V1qj+v4cwfleb/agNGRCvDZ7H6p++ansWOVOWeswp0jafRsLViYrVU7d+qNv2Qq2Z6bnq28ddlaMDdd8x6f5t5WwY4bujqcqjI92duqjediKypvWKUfHJxoqtZ54Z0imaMp7t5nNXuMu5d93LcXasowV+OeUhX+w5RTFyp3y07t2JKrJR6975du2m3OI1QJD2drhvn3JHzMbGXeO1ThUXGKv2OYgu0+/+25hm8tU36xswwfMUoRju1Klf+nDY45AAAAAAAAwFckjr5Tz636o37yeNolefx68RJNmPgts3dcKh0W3HeaMEDdPALzcx8f0EmvfigXr9PXrGPc2ugtbitR1cceLXO69FP3CR3bMeh8Vvx0qfId/daDNXRqpnK3b9cbW7Zrx2+mKc6RXVdp9/pl2myXz67SNnOT2eD42Q3bvpEzT/FmVXrFjvVa6izbrE3nYssrUqm51KFJS7Rj507lZrWyUc8zaVqab2L/kKFKeTLX+V6279SquckKdyy4r1Fx7gKlveXYSimj4hVur87vHqH4Oz3WxK/O0HLXvkLjlbpyp2NfuU9O01BH/3trP/9YrhV22a5rWKrlb5rfGYSGWlfH8mmB1pvzAgAAAAAAAHzFNydONNWlERbWV6NGN3djTVyMTpEDo86Z+jLqom4Lb1LP/mZor7Z/epdOvG2Gl0Cn0Ver9/cGWEc6qRNPv6+Tb581z1hujVTYwxHyM0MdKtbncw/JY4vLZ2q28ubEO8Le4tybNelnzmkvg6co86epGhPjWmVuVBZp4zNpSvuTR3Oagda2v25m21MVKnz193o0fY1p6zJNy16drXjHgdfplm9lOGadzvNcW87FkvDIs5r/wFCFular712pWx5YbAZOKdmvarbjYEVaf/MkuY8WoTFz0pU6Pl7hjQ5XYx1vc6Pjhc9ZrZypcQqwg/i1P9KkJ/PNM7Y4Tc7IVOrXot0r522nSpX/4jJlLNrgvkFvq6+hp1St2PKQhpq2PoV/Gq+UhY1uNgwAAAAAAP4lxH17nmbdE6+4mAgFq0pln36i3ZuWa/kf8pvJFM4nTpMfn637brpWEf2C1bXmc5UV79Xm9cu1NLelZsURGjNzlu5LiNO1A93HL9y2XkuzPfIPI+XJ1ZowqKsZNadKu/97ujJeMkMjfMw8pc+IV18z/mTTJD3qda+/CMV/Z4Zm3DOsVeeBy8NubWOvkre9uPYvKvzgA0fdXmF9++r7P/g3R/3LJzOt/b3vqNGU/3NFpmqq7sFoU7l1THB/TX/1zoiSq0u5yg/o8zkHL1lw7g7tXRqH913UbdFN6hlmhjqh4+nv6fR+M/RVdu/1G8Mc163m8wJtzj9fMBynhHHRztXfJ4q08fWW/jFvpzadi3U2o8cqShdxHoMTNSbKfjdnVPHuZuWbVfGNhccnW+/c+j+wFs/H+j+Nrw5VqH3iF7wuX/A1BAAAAAAAXzJxmpH9lFJdP9tvpGbfOs2dkqFWNZ0YPVsr0qZpaLO7qlHF60s1/ccrvQPwgdO05LepSujfkLp5qTm0VUv/7RGt8shV0v+yU/dFmUGzqpS/6C7NWm2G9pcJv1qgHydFuLM9i/eC1ETN+2OmJsc2WhRp1By0zmOW93ng8vAM7i9F0H6p9/dldmUE9/fGqP/9Dam5arcX6Miy02Z0cZqG9pbqclWm79MZ181FLX6zhipsZDczkk7/+S0d/7sZAAAAAAAAAG2UkJWrJV9133dPNTWqqZMCursj7qr8xZo6s1Hg3sQELXk5XQmu0L6uRhUlparqFqGohlDebiE8V5N+ttWMIzT7jzlKiTXP11WpbH+5tVWAQq+JaOhAYH958MiUDDn7FIy1jpPpPk6zPIL721K1LH2a4gc0/WLAM7hPWfqqZt/mDu2rDhapQmGKGuCeq9m7UpMeWMzK+8uM4L7jtDW475Dm7n5XB5nKVqe6Dy9vaG+r//CYPL+x8L/aHeIDAAAAAAAAbTJwnmbd5Qrta1T8UprGjxypO0aNVEp2gczd9xQcP03zxptBS+ZMdofpNUVaP3ek/te3JmnSuJF6JLfI2rstQFFffUgzHLVthhJdob0qlL9wqsZPsV4zZbySntxqzTgFxAyT+86AYQpwvaQqX4tvvlm3NHl4rLb/ylDFmdC+at+GhvsEektV8o2ugL5K+Uus43/dOo+v36XxS/IbrkNAbKIeGmgGAJrokODev69nSH5GdQdN2Vj/rvJr6IN/fm0J7R0O1sjjFrXq4nVOAAAAAAAAQOuFz4hXnOu+ehVva2W6u4974TNp2rjXDBSqoeOmmbp5Kde5V+2XvZ6ljNfNwLLtZ1na6srSug9V8hxTe6r6RNtyPNoI5+7QJ67E3EuU+roy9spybTbledVVqGB1mqZOSVN5vZnzdFu0wsw9AFVVqG1/cJ9H2R/Wq8D1DYLCFM79TIEWdUhw7+206pv7BUX/nuo5f7jCMmLU9QLhfZtDe9v7Z9Tcvy0AAAAAAABAW02IdoftVR9v03pTO5Vq5T53gB18VazCTX0hNaecTW3c8lV1xpSWiK9MMdW7KnWF4sFxSviO+3zCv5OgOFdAX1GqAlN6qT+jnuNTtWB5jnLW5GhVdqZSx8eZJ42PN2vp3K9pxqLz3Fz2rbka71qtf9dMrTLTTtEKblg7e1wVHl9IAPDWAcF9Z3XyStib4Qjtr1c3+x+UwDCFnCe8b1do35wu1nmZEgAAAAAAAGiL0GB33/eKQ2tM5VZWWNbQJkYhYUo2ZXPKTzub4diiYud5h/y3pWtYpKltnbuaYp2yVm1VheOlwYr/4WrlrrFD+Fyt/mG8NWOpqdC2VQvcXypMjVBDR56QUVrxxENKHh6tqJhoxcWP1YwnVut/lk5zH/+tNXrhIsL2hKwxGupajV+cr/XcnBZoUQcE92d1rtaULegUGyJ/17eAthbC+0sW2ttqrfMyJQAAAAAAANBRNuZ90NCTXrFT9OzKdM0YN1ZjpmcqJ2uColwteRop+8MjemTJBhVWWgP/YIXH2CG8uTFtZZE2/+5HesSjdY38zF9LQEioAqoqVLyvSGX2643Q21L11OPxZtR+CU/kaKHrxr123/4lrhvkAmiOD7TK6Sa/waY0zr1+QEd/f1Be+X6j8P6iQ/vBXT3/bQIAAAAAAAB8Q+4Crcx3N6UPHTxBqRmZynx4rKKCq5Sf7xG+N4hQym9e1oq5YxUX4pypOlSk4kNmPyHRSn640Qr6vOXKSF+qF17arG0vLdb4u76mSVMmafzd47W44fgBirozxeOGtm3nCO3HR1t7slnnv+wRr779AJrqkOC+7vPTprJ1lf8AU3o4X3jf7RuXYKX9gAB5fjlZ63VOAAAAAAAAQBt43EwxoHszK9T7Bzvb1disbT3a1DejVKtmTlXa6nyVedxUtqaySBsXztTmsz3NjDV3utxZfD9TqaNDneF4jbXd4+OVNG6SJo27S+Mf36Bi033HXkGf+X1nrU/ztfml5cpKn6tH0ld69K23j79OhXVmGBqh9q25j9DkrFzv0D57pmZ5rvoH0KwOCe7rD5w0lc1f/tc13JXCS0vhfc/JF98ex++6Xl497esOENwDAAAAAACgffIb7gwrhQ9MbnLz2ZTr3DeL1aFCvWDKlpVq46KZGn+XudGr9bjj7klK+1OwhoW7vgKoUeneDY4q5WZXOC5V7V6vtI3ucLxsY5rW7XavoI9LSDX1+VSo6qLiMvsXAM/qx1+NcJ5XXYXyl0zVrGcKHc8COL+OaZWz54T1z4pbl0EhLZ5Is+G9p3b1tO+igEGeXxacUO0eUwIAAAAAAABt5N2XPkGzR5vaNjBVyUPcN3QsfMczto9Q/FeTFT/QDC3hT+Rox86dzsffFirBzNvCv/OQElzbnirU1mxTewjuGd7oi4MIRfR0Hz8gwBnxp2S/2nCc/3lqgmOuwdQExblecqrK/d5aJU4zsldb18D8AsAO7f9rOivtgTbomOB+/xFVewbtYf3V/VZTN6PF8L69N6K9tb+CwkxtO1SuM/tNDQAAAAAAALRV7iptLTa1IpSclatlj6cqde4S5ax8SEO7m6eq8rVxoSvAjlf6mhwty1qoZdbfdBP2l63YqkLXqtcByVr4t2e1wO5x/1SOVv8wvqHlTtkby7XC1Js/LHUvlI2doOxfTXN+GTAwXikZSzQm1vmUzfXFwaqdRQ2vCU2YrZxfpWryuLGa/PAS5T7sPk7Ve5u11NQXlqjZ/52t1Hj3FwUVe95WxaBZyrTfg8cjdbzZAEATHRPcq1bVb3vcnlpd1X1MH6/WNY01Ce/bG9pbb7nrmP5eN6Y98/bnOmtqAAAAAAAAoO3ylbFkXUMveQVEKH7iQ5rx7URFuTJse+X5igytMkNpjIbFmAY3AdEaluQs9eliLdvoDtUDBgxV8rixGpMQrWBz08aa4nXKmrfVObCULVqm9Xvd7XDCk2Zr2bqd2rEuW7PHWa8zz9TsW6dlri8OnknT0oab0AYrKukhzcvI1LzpiQp39d2pyNfyJ1eaQWskKfFGd2hvCx1unbt9/o0eySPMBgCa6KDgXjq37qBOeyyh7/SVqxXk+bufZjSE9+0O7S0JkQr+isdtaWsP69Q6YnsAAAAAAABcpNczNOmhLG3c53FHWaPmYL5WPdG4XUyedruS/poi7c5zlrZtP5ukGdmbVey59tVWV6Oy/JWa+60MbTNTTluV9cBUZeUUeN3QtkFVqQrWZmnGFM/X2TehnamslwpVccpMuZjjpD00U6s+NXMALptOkQOjzpn6suv84BD1vbuHGVmqP9ORH+5XbYsN7Q37zrQX2qY5XXqq539dr26BZmypeeWfOvpce3YGAAAAAAAAtCROCY6V7mdU8e5m5bcYfts97mOlvS1vEx6frKF9u1pVlYpf2qpW3d51cKLGmKX+VcUbtO19R3leDcepKVfBP/JVZubx5RF3/WD95PE0R/3LJzNV+EEr/sM4j0u9vy8z/+eKTNVU3YPRpnLr0OBeXbopaNFQ9ehlxpazB4t1JO2Q6i91lt6lqwLn36Be13istj9WqvI5JZf+WAAAAAAAAADgYwjuO05bg/sOa5XjUHtaJ1eUeC2e7zwgSn3m91Nne1X9pdJcaK+TqrKOTWgPAAAAAAAA4F9NWN++juD94h7Xm73hUuvYFfeG3/QbFJoU5HVz2nPlpTq6qES1JWaivSJ7qsecWAWFeX5HcVa1eXt05NnTZgwAAAAAAAAAX25hYX31k7Q0x99L7cGUB0yF5lxZK+6N+mf3qGL7STNy6hQWoT4ZQ9Vzcjd1as/q+y6d5Tc5WmEZ1zcK7aXa7YT2AAAAAAAAAP61lJd/rhfXrnX8vVTsfT3zu9+aES4Vn1hx79RZfrMGK3Sk98p7h9rTqt5RohNrK1V/6KyZbF6n/kEKmBiuoFv6qEuTwP+sare/r6PLTspH3jQAAAAAAAAAXFb2inu7Vc6lQF/71rmybk7bjE73XqPQSVfJz89MNHKuulZ15UdVd8pMuHQPUkBYN/kFtvAjgvrTOpXzgar+7ptN7f1j7tHkCaM0KMT04a+rVsn7L2ntiztUWeec6nhJmpZxj7TpMa183UxdkL8C+/dT4MnDqjxxed5I0E1TdP/Y4YoMdI6rD+1S7vNrVFjhHAMAAAAAAADA5XRFtsrxdO7v+1We/oGqSpoP2DsFdlGXyH7qdl2jR2RQi6H92ZISHUkv8NnQPmj0TM35bpIGntmrl3OW66mlq7V2V6lChkxU6owkBZntrkwxSvruw/rO2Bgz/oJdN1EPfXO4wip26Plnn9ZTz29SUeBwTZk+RZGe9yYGAAAAAAAAAB/lc8G9Q8lxnXrsnzq8YJ9OHTrTzrY2Z1V/6DNVLsjX54+VXvxNbr8o/iM1bvTV8v90k/7r6dV6c/c+VR58T3vWL9ei53fpxMAkTRwdbDZ2CVTQgHCF2I8ejdNoe4W7mQ/srehbR2lQhFl67nhuiG5IHK6wxq/r0U8h/e3jmNc3u++WtHQ+9nywulmVf9dg6/l+CvTa5fneR/vccMdwhZzYpT9mr9VHn5Sp8v08vfDsFh3sNVxfS3JdR+dxg+zL0uNqDUocqciG41vPXTtcI24d4ny+FfxDzXsY0Nu6es1r2CbU3qnzGjfd/6W/HgAAAAAAAACuPD7XKqdZvbopYGKoug3uo65h3dSphTY6qq9TfflxVb9/SKfXHlf9MTPvy0bP1Px7ArV58WJtbdLKJVwjvj1ecce26PkN7zmnQkdq8ve+oet7WHW99bCuxYkP/6rfrtwu5+19YzV23oMadPg9nYiIVZgdEneVDr65WrsG3K+xEVK1Pedfp482/UbPv37U8arIyT/RjGvLlF8Ro/hrpLp6f/n71eng26v1zF/3OrZptlVOw/nUqfqM81iV76/V8ud3WOfj3N79Q48q5a/8pTZ8aJXnfV17xWjsfzyk+OOb9PPf5Zk5m/OaxFds0M9/v8UaO8+ry973FBITa10N5/X44O9/1omEqYrvVq06/0Br9ojyn1+sDXtbaPHjH6vkmVOV2N9fdWesbbpaezpzQBuWZVvX0Wyj3rph2kxNvC7YuqjWNv7+qvxnnoquS1LI9uauo1U3+7kCAAAAAAAAuFK1tVWOX89eIT81te86U6f6Xcd1ZtMhnVxfqpObP9Pp7Z/pVN7BhsfJFw/oxJ+tetMR1ew6o3NnzGt9XOSt92hE0AHlbXpPx82cW5U+27NDez523eU5VmN/OEXDzu3Syl8/pfX/eEXbDofq5jvv1E0hJXrL0cQ9TIMShyumR5X+8dtFevF/XtNu/1jddXuCrjnxipYselavvvZPnRo0UrcNCta+bc7j9hwySiMGhivo8Dr97unV2rR5mz4MjFXC7fEKOfyaPnScQrSGJcdIn7yigv322Hk+Q05uUfaiZ7TpVef5xCcl6Tr/t/XOJ3tVkFei7rcOV1Dhc1rw9Dp97Ai0L/S6Gnujdjiq4MF3K7Z7mXa89bEaGiP5X6f45FiFnizWazuLrQnn+7im0wH9/tdLtSlvmw5cdYvuvt06z49XakH2Wm3Zap33TSMVP6DOvKapyPum696rj+nVVb/R6vWbtGVLiXredqdGRpzSll3On3gE3vM9fe+mrvpg/TIte/5vem3LR+py1zd1R2hnVTa6juf/XAEAAAAAAABcqTp/c7apmjq7bomp3HyzVc6FHKvX2f21Xo9zV8Lq+mYM6N24Dc55DLtdN/aqUn7uGhVVO6fq3lujtQVVChl8uwY5pxwq923VHkfeW6fK/Yd1wq9aH7+VZ1ZvH1V+YZnUo48GOMYuB7T9edfNcKt1cEOudlQG6sb4kY5nm3Ccz2G9/ucNKjeL0uveW6vNn9QpMjZRLXaaaffrpupHj/+HvjGs5TYy7xTsU13YSD1w33BnW54esUr8wVgN6uR83lPRP9ea41erqPSo5HdE/3zZ/Lqgbq/e3V8l9e6rSOdMEyUv/kq/yHxaW/e5Poy9eueTKvmHDTDXNVi3xIZLn76pF3Ycdsyo7oC2/nWXyp0jpzZ8rgAAAAAAAAC+/K7M4P5L5ODRKlO1Qq9uCtRRldutZjyUHDoqBXZTiBk3r1anT5uyJSeO6KAJ0p0OyD49/x4hzYfpjvPpp9Ez5+vRx12PxzTuan8pKFhhZrMm2vm6wH69FRLYR3372V3zW/D2c/rD20fU+5YpevSJX2j+vAd1a9Umvbzf6421oFpn2vQFUKDCho3VxO/N1o/s95D2C80YGiz5+dudbizhCg6SKo+WOUYNPj0qr0/9oj5XAAAAAAAAAF82BPcdrPz4aalHuKJ6mQkvwYq84x7FD+1nxh2krk5mIXgzjqjwze163fOxNU8vb9ntvaq8iba/rnrT0/p5+mNasel8X3bUqeSvi7XgZ7/UU0uf1lNZ87Vo5S75B/k3DdAvUuQ3f6zUSSN11ZkDevu1XK1cvlArC9rwRQwAAAAAAAAANIPgvoNVv7lbRXX9dOs3hpsZD7FjNXFskm7sb5bKf3ZEleqnq4Y4hy7REf2cq+XNuN2afIEwRFf1sc7xhKvHfiMVVTph/Tnx4Sa9+Yr7sWPXDu14Y2/LYX97X9cKN6TM16MPf0NhdVWqPFimyhN10sB7NLR/tYred91k91KI1Y3XBetEwWotXblWb27dpYMHqxTUw/PXAPtUflQKuXqI968Ibrrau0XRF/25AgAAAAAAALiiENx3tGN5Wpt3QIGxUzTne2M16NpwhQwIV/Toqfr+5OEKObZLL+eZVdwfvqk9FYEaMfZB3RBhN68JVNjtD+obgwNV/v6bct4O9WL00x1Tx2qAvWv/fopLGa8RQVV69+1dzqcbe2+LdlX00e33u87HX4HXjdV3Zs3Vj/63K4UuU9VJqUfo1QoJDJS/3Xe+Va9rnz0flsq/f7y+ed9wBQVa+x04UhPvv0Uhpdu1+T2z0SVR5mgj1CNiiPN6OT6LqUq+xrP/fp3yX9ulypBbNOPhqUpMHGV9dtbnek+M9Y49fOGfKwAAAAAAAIAriV/PXiE/NTU6SO3+XXq3qreih92mhPhbdZv1GBbTR/UHXtOK3/1VnzW0Zz+moo+Oqfv1I/XVu5J0Z/Kdir8uREcL1mnVix+o1rFNmAYlDlfo0d16633H3Wml0CG6bVgfHSnYqo/NlK65RXfGSB/n7ZDdQKbnkFEaEfSBNu0fpPvv/7ruvOt2Deldow82ZGv9btca+GgNS7Ze9MkrKthvjxufT5IShl2jzgfy9Ifn39Sps/Y2NTpwrJdGjEzQnXfeqcja16zXHm3F69qp9F190DlKt942Snfdae335lj1PLZdq363QRUN+238PiyNrofNcU16H9U7297TcTPnVqOSwwGKiR+ppKS7nZ9Fv0N6o6SbooM9XlPxnnYe6Kyrrr9eQwfFKKb3ab255l0F3BR9nuvY3OcKAAAAAAAA4ErV+ZuzTdXU2XVLTOXWKXJg1DlTwxf06KeQoNOqPHSBXun2dsFS9eeHVd2a+65eQOTkn2jGtfu0ImuNShz7rtWJg0fV6l0H9lZI726qPVqmk832uvFXYA9/VZ9o9OQFX9de1vH695PfsUu938acxwlUVcufmf0zgzrPK5mkaRn3SJse08rXzZTLJf5cAQAAAAAAAHQ8/+eKTNVU3YPRpnKjVY6vOXH4wqG9zd7u4BcU7jr23YbQ3lZ91NFTvuWQvK5paG+74OvayzreoS86tLc5j9PSZxZ010w9+n9nKjHG0U/HEqjoybcrWkd0qLn/rX6RnysAAAAAAACAKwLBPfAFOrn1Jb1xuI+Svztf8zN+YT3ma9oQ6aNX1+jlT81GAAAAAAAAAOCBVjnA5eBogWPfkvZ021oQAQAAAAAAALji0SoH8EWOFjhlbW9BBAAAAAAAAOBfDsE9AAAAAAAAAAA+hOAeAAAAAAAAAAAfQnAPAAAAAAAAAIAPIbgHAAAAAAAAAMCHENwDAAAAAAAAAOBDCO4BAAAAAAAAAPAhBPcAAAAAAAAAAPgQgnsAAAAAAAAAAHwIwT0AAAAAAAAAAD6E4B4AAAAAAAAAAB9CcA8AAAAAAAAAgA8huAcAAAAAAAAAwIcQ3AMAAAAAAAAA4EMI7gEAAAAAAAAA8CEE9wAAAAAAAAAA+BCCewAAAAAAAAAAfAjBPQAAAAAAAAAAPoTgHgAAAAAAAACAjlBeYgpvBPcAAAAAAAAAAHSAc6UfmcobwT0AAAAAAAAAAB3g3DuvmMobwT0AAAAAAAAAAJfZ2Zd+p7Nbc8zIG8E9AAAAAAAAAACXQ3mJzu3O09n/TtPZtYuk2jPmCW+dIgdGnTO1l6iYGFMBAAAAAAAAAID2Kt63z1St02JwDwAAAAAAAAAALj9a5QAAAAAAAAAA4DOk/w/F+cnlmVj9cwAAAABJRU5ErkJggg=="
    }
   },
   "cell_type": "markdown",
   "metadata": {
    "id": "0ukcJl_g78nK"
   },
   "source": [
    "![image.png](attachment:31a22949-a3a9-47d7-8b17-39f1e51b18df.png)"
   ]
  }
 ],
 "metadata": {
  "accelerator": "GPU",
  "colab": {
   "provenance": []
  },
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}